In [1]:
# ============================================
# 🔹 ESOL — Solubility Prediction with GNN (PyTorch Geometric)
# ============================================

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from rdkit import Chem
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

/home/kostanew/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================
# 1. Load Dataset
# ============================================

df = pd.read_csv("delaney-processed.csv")

# Normalize target (recommended for regression stability)
y_mean = df["measured log solubility in mols per litre"].mean()
y_std = df["measured log solubility in mols per litre"].std()
df["y_norm"] = (df["measured log solubility in mols per litre"] - y_mean) / y_std



In [3]:
# ============================================
# 2. Molecule → Graph conversion
# ============================================

def atom_features(atom):
    """Return a basic set of atom-level features."""
    return [
        atom.GetAtomicNum(),
        atom.GetTotalDegree(),
        atom.GetFormalCharge(),
        atom.GetTotalNumHs(),
        atom.GetIsAromatic() * 1.0,
    ]


def bond_features(bond):
    """Return bond type as numerical encoding."""
    bt = bond.GetBondType()
    return [
        bt == Chem.rdchem.BondType.SINGLE,
        bt == Chem.rdchem.BondType.DOUBLE,
        bt == Chem.rdchem.BondType.TRIPLE,
        bt == Chem.rdchem.BondType.AROMATIC,
    ]


def mol_to_graph(smiles, y):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # --- Atom features ---
    x = torch.tensor([atom_features(a) for a in mol.GetAtoms()], dtype=torch.float32)

    # --- Bond connections ---
    src, dst, e_feat = [], [], []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        src += [i, j]
        dst += [j, i]
        b = bond_features(bond)
        e_feat += [b, b]

    edge_index = torch.tensor([src, dst], dtype=torch.long)
    edge_attr = torch.tensor(e_feat, dtype=torch.float32)

    y = torch.tensor([y], dtype=torch.float32)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)



In [4]:
# ===========================================================
# 3. Custom PyG dataset
# ===========================================================

class EsolDataset(InMemoryDataset):
    def __init__(self, dataframe, transform=None, pre_transform=None):
        super().__init__(".", transform, pre_transform)
        data_list = []
        for smiles, target in zip(dataframe["smiles"], dataframe["y_norm"]):
            g = mol_to_graph(smiles, target)
            if g is not None:
                data_list.append(g)
        self.data, self.slices = self.collate(data_list)

    def __len__(self):
        return self.data.y.size(0)

In [5]:
# ============================================
# 4. Train/test split + loaders (✅ compatible fix)
# ============================================

dataset = EsolDataset(df)
train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)

# --- manually rebuild subsets (safe across PyG versions)
train_dataset = [dataset.get(i) for i in train_idx]
test_dataset = [dataset.get(i) for i in test_idx]

from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

train_dataset = dataset.index_select(torch.tensor(train_idx))
test_dataset = dataset.index_select(torch.tensor(test_idx))


/tmp/ipykernel_144874/1775182624.py:16: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  return self.data.y.size(0)


In [6]:
# ===========================================================
# 5. Model definition (GCNConv-based MPNN)
# ===========================================================

class MPNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = nn.Linear(hidden_channels, hidden_channels // 2)
        self.lin2 = nn.Linear(hidden_channels // 2, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        return self.lin2(x).squeeze(-1)  # ✅ fix shape mismatch


In [7]:
# ===========================================================
# 6. Setup training components
# ===========================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MPNN(in_channels=5, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=20, factor=0.7)
loss_fn = nn.MSELoss()

In [8]:
# ===========================================================
# 7. Train / Evaluate functions
# ===========================================================

def train_one_epoch(loader):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(loader):
    model.eval()
    preds, targets = [], []
    for data in loader:
        data = data.to(device)
        pred = model(data)
        preds.append(pred.cpu())
        targets.append(data.y.cpu())
    preds = torch.cat(preds).squeeze()
    targets = torch.cat(targets).squeeze()
    mse = mean_squared_error(targets, preds)
    rmse = mse ** 0.5
    r2 = r2_score(targets, preds)
    return rmse, r2

In [9]:
# ===========================================================
# 8. Training loop
# ===========================================================

best_rmse = float("inf")
for epoch in range(1, 5001):
    train_loss = train_one_epoch(train_loader)
    rmse, r2 = evaluate(test_loader)
    scheduler.step(rmse)

    if rmse < best_rmse:
        best_rmse = rmse
        torch.save(model.state_dict(), "best_esol_model.pt")

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | TrainLoss: {train_loss:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")

print("\n✅ Training complete! Best normalized RMSE:", best_rmse)

Epoch 010 | TrainLoss: 0.7373 | RMSE: 0.9097 | R²: 0.2305
Epoch 020 | TrainLoss: 0.5616 | RMSE: 0.7695 | R²: 0.4495
Epoch 030 | TrainLoss: 0.4421 | RMSE: 0.7514 | R²: 0.4750
Epoch 040 | TrainLoss: 0.4204 | RMSE: 0.6241 | R²: 0.6379
Epoch 050 | TrainLoss: 0.3840 | RMSE: 0.6033 | R²: 0.6616
Epoch 060 | TrainLoss: 0.3352 | RMSE: 0.5984 | R²: 0.6671
Epoch 070 | TrainLoss: 0.3645 | RMSE: 0.6115 | R²: 0.6523
Epoch 080 | TrainLoss: 0.3158 | RMSE: 0.5712 | R²: 0.6966
Epoch 090 | TrainLoss: 0.3060 | RMSE: 0.6352 | R²: 0.6248
Epoch 100 | TrainLoss: 0.4030 | RMSE: 0.5899 | R²: 0.6764
Epoch 110 | TrainLoss: 0.2929 | RMSE: 0.5664 | R²: 0.7017
Epoch 120 | TrainLoss: 0.2908 | RMSE: 0.5607 | R²: 0.7076
Epoch 130 | TrainLoss: 0.2855 | RMSE: 0.5505 | R²: 0.7182
Epoch 140 | TrainLoss: 0.2755 | RMSE: 0.5424 | R²: 0.7265
Epoch 150 | TrainLoss: 0.3028 | RMSE: 0.5392 | R²: 0.7296
Epoch 160 | TrainLoss: 0.2832 | RMSE: 0.5394 | R²: 0.7294
Epoch 170 | TrainLoss: 0.2608 | RMSE: 0.5462 | R²: 0.7226
Epoch 180 | Tr

KeyboardInterrupt: 

In [10]:
# ===========================================================
# 9. Evaluate best model (denormalized)
# ===========================================================

model.load_state_dict(torch.load("best_esol_model.pt"))
model.eval()

preds, targets = [], []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        pred = model(data)
        preds.append(pred.cpu())
        targets.append(data.y.cpu())

preds = torch.cat(preds).squeeze().numpy() * y_std + y_mean
targets = torch.cat(targets).squeeze().numpy() * y_std + y_mean

rmse_final = mean_squared_error(targets, preds) ** 0.5
r2_final = r2_score(targets, preds)

print(f"\n📊 Final (Denormalized) Performance: RMSE={rmse_final:.3f} | R²={r2_final:.3f}")


📊 Final (Denormalized) Performance: RMSE=0.957 | R²=0.806


/tmp/ipykernel_144874/312461329.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_esol_model.pt"))
